## [Create](https://docs.databricks.com/en/sql/language-manual/sql-ref-syntax-ddl-create-sql-function.html) and register `scientific2simple` as a [`UDF`](https://docs.databricks.com/en/udf/unity-catalog.html) to Unity Catalog 
To democratize the usage or our [Classified Proteins]($./02_TransformerCNN_Protein_Classification) in our [AI/BI Genie](https://www.databricks.com/product/ai-bi/genie) [Space](https://docs.databricks.com/en/genie/index.html), we can leverage [Foundational Models API](https://docs.databricks.com/en/machine-learning/foundation-models/index.html#use-foundation-apis) to help simplify _protein scientific terms_ and provide _layman simple terms with definition_.

To do so, we will use [`ai_query()`](https://docs.databricks.com/en/sql/language-manual/functions/ai_query.html) [to query the Foundation Model](https://docs.databricks.com/en/large-language-models/how-to-ai-query.html) `databricks-meta-llama-3-3-70b-instruct` (or any other suitable LLMs available as FMAPI on Databricks e.g. `databricks-claude-sonnet-4-5`) in a specific way to bulk convert the `OrganismName` scientific terms to layman simple terms and to provide [`UDF`](https://docs.databricks.com/en/udf/unity-catalog.html) [structured outputs](https://docs.databricks.com/en/machine-learning/model-serving/structured-outputs.html). 

To reuse the logic of converting scientific terms to simpler layman terms (e.g. as a tool in [function-calling](https://docs.databricks.com/en/machine-learning/model-serving/function-calling.html)), we can register the the `ai_query()` as a [`UDF function`](https://docs.databricks.com/en/udf/unity-catalog.html) and then use it in a query to create the desired table with simplified terms and their meanings.

We'll walk through how this can be achieved.

_NB: we can use a serverless compute for setting this up._ 

<!-- 
https://docs.databricks.com/en/genie/trusted-assets.html

https://docs.databricks.com/en/sql/language-manual/sql-ref-syntax-ddl-create-function.html

https://docs.databricks.com/en/udf/unity-catalog.html
 -->

In [0]:
%run ./utils

### [1] Let's review our classified proteins data: `proteinclassification_tiny` 


In [0]:
remove_widgets() 
uc_config = setup_uc_paths(spark=None, use_widgets=False); ## if you update the values in widgets -- it will automatically trigger an update of the UC paths

# Extract catalog, schema, volume names
catalog_name = uc_config["catalog_name"]
schema_name = uc_config["schema_name"]
volume_name = uc_config["volume_name"]

In [0]:
sDF = spark.table(f"{catalog_name}.{schema_name}.proteinclassification_tiny")

In [0]:
display(sDF.sort("OrganismName"))

# display(sDF.groupby('OrganismName', 'ProteinName', 'ID').count().sort('OrganismName'))

We can see that some Organisms are associated with different types of (e.g soluble / membrane transport) proteins as denoted by their corresponding `ProteinNames`, `OrganismIdentifier`, `GeneName`, and `IDs`

In [0]:
from pyspark.sql import functions as F, types as T

display(sDF.groupby('OrganismName').agg((F.size(F.collect_set('ProteinName')).alias('N_uniqueProteins')),
                                        F.collect_set('ProteinName'),                                          
                                        F.collect_set('ID')
                                       ).sort('N_uniqueProteins', ascending=False))

It would be helpful if we could convert these scientific `OrganismName` into simple layman terms so that non-SME biologists can more easily explore the data. 

We will write out the unique set of `OrganismName` as a separate Delta Table to our Unity Catalog to use for exploring the conversion of scientific to simple terms using LLMs. The derived simple terms can then be linked by by the scientific `OrganismName` later. 

In [0]:
## Uncomment to write out the table
sDF.select('OrganismName').distinct().write.mode("overwrite").option("mergeSchema", "true").saveAsTable(f"{catalog_name}.{schema_name}.tinysample_organism_info")

## check/read tinysample_organism_info 
sDF_orginfo = spark.table(f"{catalog_name}.{schema_name}.tinysample_organism_info")
display(sDF_orginfo)

### [2] Use `ai_query()` to leverage LLM to help simplify scientific terms

We can query and look up these scientific `OrganismName` with the help of LLMs and try to organize the query output in a way that will facilitate our use of the corresponding protein dataset. 

Let's use the [Foundation Model](https://docs.databricks.com/en/large-language-models/how-to-ai-query.html) `databricks-meta-llama-3-1-70b-instruct` model and guide it to help our case, by providing a detailed prompt. We will wrap it in an [`ai_query()`](https://docs.databricks.com/en/sql/language-manual/functions/ai_query.html) that request that it provides a [structured output](https://docs.databricks.com/en/machine-learning/model-serving/structured-outputs.html) JSON response:


e.g. 

| Scientific `OrganismName` | `SimpleTermDict` (JSON Response) | 
|-----------------|---------------|
| Danio rerio     | {"simple_term": "Zebrafish", "meaning": "A species of freshwater fish used as a model organism in scientific research, particularly in the fields of developmental biology and neurology."} | 


<!-- | Scientific `OrganismName` | `SimpleTermDict` (JSON Response) | `simple_term` | `meaning` |
|-----------------|---------------|-------------|---------|
| Danio rerio     | {"simple_term": "Zebrafish", "meaning": "A species of freshwater fish used as a model organism in scientific research, particularly in the fields of developmental biology and neurology."} | Zebrafish | A species of freshwater fish used as a model organism in scientific research, particularly in the fields of developmental biology and neurology. | -->

In [0]:
# Use the ai_query function directly in a query and perform the necessary scientific2simple term transformations in a single query

organism_info_sDF = spark.sql(
    f"""
    SELECT
      OrganismName,
      SimpleTermDict,
      get_json_object(SimpleTermDict, '$.simple_term') AS Organism_SimpleTerm,
      get_json_object(SimpleTermDict, '$.meaning') AS Organism_Definition
    FROM (
      SELECT
        OrganismName,
        ai_query(
          'databricks-meta-llama-3-3-70b-instruct',
          CONCAT(
            'As a knowledgeable and factual encyclopedia you respond succinctly. Provide a dictionary of responses in the format {{"key": "response"}} to the following keys a "simple layman term" and "meaning" for the given "scientific term": ',
            OrganismName,
            '. Output just the dictionary {{"simple_term": "response", "meaning": "response"}}.'
          )
        ) AS SimpleTermDict
      FROM {catalog_name}.{schema_name}.tinysample_organism_info
    ) ORDER BY OrganismName
    """
)
display(organism_info_sDF)

### [3] `ai_query()` structured output JSON parsed: 

Our [`ai_query()`](https://docs.databricks.com/en/sql/language-manual/functions/ai_query.html) outputs the keys `simple_term` and `meaning` of the scientific `OrganismName` and their corresponding values, which we extract using `get_json_object` parsing within the single SQL query. 

**To reuse the logic of converting scientific terms to simpler layman terms (e.g. as a tool in [function-calling](https://docs.databricks.com/en/machine-learning/model-serving/function-calling.html)), we can**    
**-A. register the the `ai_query()` as a [`UDF function`](https://docs.databricks.com/en/udf/unity-catalog.html) and then**    
**-B. use it in a query to create the desired table with simplified terms and their meanings.**  

(_This is particularly relevant given that we cannot directly include `get_json_object` parsing within the `SQL function` definition itself, we can create a view or a separate query that uses the UC registered `sql function` and then apply `get_json_object` to parse the JSON response._)


#### [3A] Define an `ai_query()` to process `scientific2simple_dict` as `UDF function` + register to UC

You can use SQL and pyspark.sql to define and register the UDF function to Unity Catalog.  

In [0]:
### via PySpark 
# Define the SQL function
create_function_query = f"""
CREATE OR REPLACE FUNCTION {catalog_name}.{schema_name}.scientific2simple_dict(OrganismName STRING)
RETURNS STRING
COMMENT 'Returns a dictionary of simple terms and definitions for the given scientific term as JSON output.'
LANGUAGE SQL
RETURN ai_query(
  'databricks-meta-llama-3-3-70b-instruct',
  CONCAT('As a knowledgeable and factual encyclopedia you respond succinctly. Provide a dictionary of responses in the format {{"key": "response"}} to the following keys a "simple layman term" and "meaning" for the given "scientific term": ', OrganismName, '. Output just the dictionary {{"simple_term": "response", "meaning": "response"}}.')
);
"""

# Execute the query to create the function
spark.sql(create_function_query)

#### [3B] Use the registered SQL Function in a Query 
Additionally, perform necessary transformations to extract the simplified scientific `OrganismName` and definitions, then write out transformed data to UC. 

NB: You can definitely leverage available LLMs to simply extract everything (human layman terms and any other downstream probe on the data) at once in an interactive query in genie but if these are common queries on a bulk of data it would be most efficient and cost-effective to preprocess the joins prior. 

In [0]:
# Use a Common Table Expression (CTE) or a subquery to ensure that the ai_query function is called only once per OrganismName

orginfo_sDF = spark.sql(f"""
WITH SimpleTermData AS (
  SELECT
    OrganismName,
    {catalog_name}.{schema_name}.scientific2simple_dict(OrganismName) AS SimpleTermDict
  FROM {catalog_name}.{schema_name}.tinysample_organism_info
)
SELECT
  OrganismName,
  SimpleTermDict,
  get_json_object(SimpleTermDict, '$.simple_term') AS Organism_SimpleTerm,
  get_json_object(SimpleTermDict, '$.meaning') AS Organism_Definition
FROM SimpleTermData
-- ORDER BY OrganismName; -- this slows down the query
""")

# writing out the query to a UC Delta table speeds up subsequent queries and can be used for incremental updates
orginfo_sDF.write.mode("overwrite").option("mergeSchema", "true").saveAsTable(f"{catalog_name}.{schema_name}.tinysample_organism_info_scientificNsimple")

### [4] Read and display the resulting saved table with simplified `OrganismName`

In [0]:
orginfo_sDF = spark.table(f"{catalog_name}.{schema_name}.tinysample_organism_info_scientificNsimple")

display(orginfo_sDF)